<a href="https://colab.research.google.com/github/hananbahtiti/Hybrid-Intrusion-detection-Systems/blob/main/NetworkPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.1 MB/s eta 0:00:00


In [35]:
!cp /content/eve_del.csv /content/drive/MyDrive/hybrid_IDS/dataset/

In [21]:
df = pd.read_csv('/content/drive/MyDrive/hybrid_IDS/dataset/eve_encoded.csv')
df['alert'].info()
#df.head(3)
df['alert'].value_counts()

<class 'pandas.core.series.Series'>
RangeIndex: 286150 entries, 0 to 286149
Series name: alert
Non-Null Count  Dtype 
--------------  ----- 
5100 non-null   object
dtypes: object(1)
memory usage: 2.2+ MB


<ipython-input-21-9742c755c1c7>:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/hybrid_IDS/dataset/eve_encoded.csv')


,count
alert,
"{'action': 'allowed', 'gid': 1, 'signature_id': 2010937, 'rev': 3, 'signature': 'ET SCAN Suspicious inbound to mySQL port 3306', 'category': 'Potentially Bad Traffic', 'severity': 2, 'metadata': {'confidence': ['Medium'], 'created_at': ['2010_07_30'], 'signature_severity': ['Informational'], 'updated_at': ['2019_07_26']}}",1992
"{'action': 'allowed', 'gid': 1, 'signature_id': 2010935, 'rev': 3, 'signature': 'ET SCAN Suspicious inbound to MSSQL port 1433', 'category': 'Potentially Bad Traffic', 'severity': 2, 'metadata': {'confidence': ['Medium'], 'created_at': ['2010_07_30'], 'signature_severity': ['Informational'], 'updated_at': ['2019_07_26']}}",595
"{'action': 'allowed', 'gid': 1, 'signature_id': 2002992, 'rev': 7, 'signature': 'ET SCAN Rapid POP3 Connections - Possible Brute Force Attack', 'category': 'Misc activity', 'severity': 3, 'metadata': {'confidence': ['Medium'], 'created_at': ['2010_07_30'], 'signature_severity': ['Informational'], 'updated_at': ['2019_07_26']}}",447
"{'action': 'allowed', 'gid': 1, 'signature_id': 2002994, 'rev': 7, 'signature': 'ET SCAN Rapid IMAP Connections - Possible Brute Force Attack', 'category': 'Misc activity', 'severity': 3, 'metadata': {'confidence': ['Medium'], 'created_at': ['2010_07_30'], 'signature_severity': ['Informational'], 'updated_at': ['2019_07_26']}}",363
"{'action': 'allowed', 'gid': 1, 'signature_id': 2010939, 'rev': 3, 'signature': 'ET SCAN Suspicious inbound to PostgreSQL port 5432', 'category': 'Potentially Bad Traffic', 'severity': 2, 'metadata': {'confidence': ['Medium'], 'created_at': ['2010_07_30'], 'signature_severity': ['Informational'], 'updated_at': ['2019_07_26']}}",300
"{'action': 'allowed', 'gid': 1, 'signature_id': 2200075, 'rev': 2, 'signature': 'SURICATA UDPv4 invalid checksum', 'category': 'Generic Protocol Command Decode', 'severity': 3}",241
"{'action': 'allowed', 'gid': 1, 'signature_id': 2200074, 'rev': 2, 'signature': 'SURICATA TCPv4 invalid checksum', 'category': 'Generic Protocol Command Decode', 'severity': 3}",200
"{'action': 'allowed', 'gid': 1, 'signature_id': 2032326, 'rev': 2, 'signature': 'ET EXPLOIT DD-WRT UPNP Unauthenticated Buffer Overflow (CVE-2021-27137)', 'category': 'Attempted Administrator Privilege Gain', 'severity': 1, 'metadata': {'attack_target': ['Networking_Equipment'], 'confidence': ['High'], 'created_at': ['2021_03_25'], 'cve': ['CVE_2021_27137'], 'deployment': ['Internal', 'Perimeter'], 'performance_impact': ['Low'], 'signature_severity': ['Major'], 'updated_at': ['2024_03_08']}}",116
"{'action': 'allowed', 'gid': 1, 'signature_id': 2200036, 'rev': 2, 'signature': 'SURICATA TCP option invalid length', 'category': 'Generic Protocol Command Decode', 'severity': 3}",100


In [29]:
df = pd.read_csv('/content/eve_del.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286150 entries, 0 to 286149
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   flow        284361 non-null  object 
 1   dest_ip     286148 non-null  object 
 2   src_port    285848 non-null  float64
 3   dest_port   285848 non-null  float64
 4   proto       286148 non-null  object 
 5   src_ip      286148 non-null  object 
 6   event_type  286150 non-null  object 
 7   tcp         270253 non-null  object 
 8   flow_id     286048 non-null  float64
 9   Date        286150 non-null  int64  
 10  hour        137399 non-null  float64
 11  minute      137399 non-null  float64
 12  second      137399 non-null  float64
dtypes: float64(6), int64(1), object(6)
memory usage: 28.4+ MB


In [ ]:
'tcp' , 'flow'

In [34]:
df['flow'].value_counts()

,count
flow,
"{'pkts_toserver': 1, 'pkts_toclient': 0, 'bytes_toserver': 186, 'bytes_toclient': 0, 'start': '2016-03-14T12:58:39.946046+0200', 'src_ip': '175.45.176.2', 'dest_ip': '10.40.85.32', 'src_port': 18787, 'dest_port': 32472}",2
"{'pkts_toserver': 1, 'pkts_toclient': 0, 'bytes_toserver': 62, 'bytes_toclient': 0, 'start': '2016-03-14T12:53:22.979217+0200', 'src_ip': '175.45.176.1', 'dest_ip': '10.40.85.32', 'src_port': 1024, 'dest_port': 4096}",2
"{'pkts_toserver': 1, 'pkts_toclient': 0, 'bytes_toserver': 62, 'bytes_toclient': 0, 'start': '2016-03-11T06:17:57.852848+0200', 'src_ip': '175.45.176.1', 'dest_ip': '10.40.85.32', 'src_port': 1024, 'dest_port': 4096}",2
"{'pkts_toserver': 1, 'pkts_toclient': 0, 'bytes_toserver': 121, 'bytes_toclient': 0, 'start': '2016-03-12T08:19:54.090135+0200', 'src_ip': '175.45.176.3', 'dest_ip': '10.40.85.32', 'src_port': 29177, 'dest_port': 29129}",2
"{'pkts_toserver': 1, 'pkts_toclient': 0, 'bytes_toserver': 118, 'bytes_toclient': 0, 'start': '2016-03-12T05:48:55.075828+0200', 'src_ip': '175.45.176.1', 'dest_ip': '10.40.85.32', 'src_port': 29605, 'dest_port': 58859}",2
...,...
"{'pkts_toserver': 4, 'pkts_toclient': 0, 'bytes_toserver': 248, 'bytes_toclient': 0, 'start': '2016-03-12T05:55:56.377999+0200', 'end': '2016-03-12T05:56:11.821139+0200', 'age': 15, 'state': 'new', 'reason': 'timeout', 'alerted': False}",1
"{'pkts_toserver': 4, 'pkts_toclient': 0, 'bytes_toserver': 248, 'bytes_toclient': 0, 'start': '2016-03-12T05:58:34.413321+0200', 'end': '2016-03-12T05:58:49.971013+0200', 'age': 15, 'state': 'new', 'reason': 'timeout', 'alerted': False}",1
"{'pkts_toserver': 4, 'pkts_toclient': 0, 'bytes_toserver': 248, 'bytes_toclient': 0, 'start': '2016-03-12T05:32:45.219512+0200', 'end': '2016-03-12T05:33:00.738873+0200', 'age': 15, 'state': 'new', 'reason': 'timeout', 'alerted': False}",1


In [ ]:
 df['Date'].value_counts(),df['Date'].unique()

(Date
 3    148751
 1     69401
 2     54913
 0     13085
 Name: count, dtype: int64,
 array([0, 1, 2, 3]))

In [ ]:
df['Date']

,Date
0,0
1,0
2,0
3,0
4,0
...,...
286145,3
286146,3
286147,3
286148,3


In [37]:
import pandas as pd
import numpy as np
import os
import glob
import math
import ast
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [ ]:
class Preprocessing:
  def __init__(self):
    pass

  def clean_and_save(self, file_path: str, output_path: str, threshold=80):
    try:
        df = pd.read_csv(file_path)
        total_rows = len(df)
        missing_ratios = (df.isnull().sum() / total_rows) * 100

        columns_to_drop = [col for col, ratio in missing_ratios.items() if ratio > threshold]

        print(f"Columns to drop: {columns_to_drop}")

        df = df.drop(columns=columns_to_drop, axis=1)
        df.to_csv(output_path, index=False)
        return f"Saved cleaned data to {output_path}"
    except Exception as e:
      return f"An error occurred: {e}"




  def data_encoding(self, file_path, column_number : int):
    encoder = LabelEncoder()
    tqdm.pandas()
    try:

      df = pd.read_csv(file_path)
      df.iloc[:, column_number] = df.iloc[:, column_number].astype(str).fillna(-1)

      encoder.fit(df.iloc[:, column_number])
      df.iloc[:, column_number] = df.iloc[:, column_number].progress_apply(
          lambda x:encoder.transform([x])[0] if pd.notna(x) else -1 )

      df.to_csv(file_path.replace(".csv", "_encoded.csv"), index=False)
      return f"done file..."

    except Exception as e:
      return f"An error occurred: {e}"


  def expand_dict_columns(self, file_path, column_names: list, output_file_path: str = None):
        try:
            df = pd.read_csv(file_path)

            all_keys = set()

            # المرحلة 1: اجمع جميع المفاتيح لكل الأعمدة المطلوبة
            for column_name in column_names:
                if column_name not in df.columns:
                    continue  # تجاهل أي عمود غير موجود

                # استخدام apply بدلاً من iterrows لتسريع العملية
                def extract_keys(value):
                    if pd.notna(value):
                        try:
                            return ast.literal_eval(value).keys()
                        except (SyntaxError, ValueError):
                            return []
                    return []

                # اجمع كل المفاتيح
                all_keys.update(df[column_name].apply(extract_keys).explode().dropna().unique())

            # أضف الأعمدة الجديدة (المفاتيح)
            for key in all_keys:
                if key not in df.columns:
                    df[key] = np.nan

            # المرحلة 2: عبئ القيم من كل الأعمدة المطلوبة باستخدام apply
            def expand_dict_column(value, column_name):
                if pd.notna(value):
                    try:
                        dictionary = ast.literal_eval(value)
                        return dictionary.get(column_name, np.nan)
                    except (SyntaxError, ValueError):
                        return np.nan
                return np.nan

            # استخدم apply لتوسيع القيم في الأعمدة الجديدة
            for column_name in column_names:
                if column_name not in df.columns:
                    continue
                df[column_name].apply(lambda x: expand_dict_column(x, column_name))

            # حذف الأعمدة الأصلية بعد التوسيع (اختياري)
            df = df.drop(columns=column_names, axis=1)

            # حفظ الملف الجديد
            output_file_path = output_file_path or file_path.replace(".csv", "_expanded.csv")
            df.to_csv(output_file_path, index=False)

            return f"✅ تم توسيع الأعمدة وحفظ الملف بنجاح في: {output_file_path}"

        except Exception as e:
            return f"❌ حدث خطأ: {e}"


a = Preprocessing()
#a.data_encoding('/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv',
#                column_number=19)

#a.clean_and_save(file_path='/content/eve_encoded.csv',
#                 output_path='/content/eve_del.csv', threshold=80)
a.expand_dict_columns( file_path='/content/eve_del.csv',
                     column_names=['tcp' , 'flow'])

In [ ]:
def expand_dict_column(self, file_path, column_name: str):
    try:
        df = pd.read_csv(file_path)

        # 1. Extract all unique keys from the dictionaries in the column
        all_keys = set()
        for index in df.index:
            if pd.notna(df.loc[index, column_name]):
                try:  # Handle potential errors in ast.literal_eval
                    dictionary = ast.literal_eval(df.loc[index, column_name])
                    all_keys.update(dictionary.keys())
                except (SyntaxError, ValueError):
                    pass  # Skip rows with invalid dictionary strings

        # 2. Pre-fill new columns with NaN to accommodate missing keys
        for key in all_keys:
            df[key] = np.nan

        # 3. Populate the new columns with values from the dictionaries
        for index in df.index:
            if pd.notna(df.loc[index, column_name]):
                try:
                    dictionary = ast.literal_eval(df.loc[index, column_name])
                    for key, value in dictionary.items():
                        df.loc[index, key] = value
                except (SyntaxError, ValueError):
                    pass  # Skip rows with invalid dictionary strings

        # 4. Drop the original dictionary column
        df = df.drop(columns=[column_name])

        df.to_csv(file_path, index=False)
        return "done file..."

    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
df['flow']

,flow
0,NaN
1,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte..."
2,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte..."
3,NaN
4,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte..."
...,...
286145,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte..."
286146,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte..."
286147,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte..."
286148,"{'pkts_toserver': 1, 'pkts_toclient': 0, 'byte..."


In [ ]:
file_path = '/content/drive/MyDrive/hybrid_IDS/dataset/eve_encoded.csv'
df = pd.read_csv(file_path)
#df = df.drop(df.iloc[:,11], axis=1)
df.info()
df.loc[5, 5]

<ipython-input-19-53841a2ac594>:2: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
class NetworkPreprocessing:
  def __init__(self):
    pass


  def extract_tcp_option(self, option, key):
    for opt in option:
      if opt[0] == key:
        return opt[1]
    return None


  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."

  def convert_pcap_csv(self, pcap_file, csv_file):
    try:
      print("Countig packages...")
      with PcapReader(pcap_file) as count_reader:
        total_packets = sum(1 for _ in count_reader)
        print(f"Total number of packages: {total_packets}")

      print("Start converting to csv file...")
      with PcapReader(pcap_file) as packets:
        with open(csv_file, mode='w', newline="") as file:
          writer = csv.writer(file)
          writer.writerow([
              "Packet Number", "Ethernet DST", "Ethernet SRC", "Ethernet Type",
              "IP Version", "IP IHL", "IP TOS", "IP Length", "IP ID", "IP Flags",
              "IP Fragmentation", "IP TTL", "IP Proto", "IP Checksum",
              "IP Src", "IP Dst", "TCP Src Port", "TCP Dst Port", "TCP Seq",
              "TCP Ack", "TCP Data Offset", "TCP Reserved", "TCP Flags",
              "TCP Window", "TCP Checksum", "TCP Urgent Pointer", "TCP Options MSS",
              "TCP Options WScale", "TCP Options NOP","Hexdump"
          ])

          for i, packet in enumerate(tqdm(packets,total=total_packets, desc="Packet processing")):
            if packet.haslayer('Ethernet'):
              eth_dst = packet['Ethernet'].dst
              eth_src = packet['Ethernet'].src
              eth_type = packet['Ethernet'].type
            else:
              eth_dst = eth_src = eth_type = None

            if packet.haslayer('IP'):
              ip_version = packet['IP'].version
              ip_ihl = packet['IP'].ihl
              ip_tos = packet['IP'].tos
              ip_len = packet['IP'].len
              ip_id = packet['IP'].id
              ip_flags = packet['IP'].flags
              ip_frag = packet['IP'].frag
              ip_ttl = packet['IP'].ttl
              ip_proto = packet['IP'].proto
              ip_chksum = packet['IP'].chksum
              ip_src = packet['IP'].src
              ip_dst = packet['IP'].dst
            else:
              ip_version = ip_ihl = ip_tos = ip_len = ip_id = ip_flags = ip_frag = ip_ttl = ip_proto = ip_chksum = ip_src = ip_dst = None

            if packet.haslayer('TCP'):
              tcp_sport = packet['TCP'].sport
              tcp_dport = packet['TCP'].dport
              tcp_seq = packet['TCP'].seq
              tcp_ack = packet['TCP'].ack
              tcp_dataofs = packet['TCP'].dataofs
              tcp_reserved = packet['TCP'].reserved
              tcp_flags = packet['TCP'].flags
              tcp_window = packet['TCP'].window
              tcp_chksum = packet['TCP'].chksum
              tcp_urgptr = packet['TCP'].urgptr
              tcp_options = packet['TCP'].options

              tcp_mss = self.extract_tcp_option(tcp_options, 'MSS')
              tcp_wscale = self.extract_tcp_option(tcp_options, 'WScale')
              tcp_nop_count = self.extract_tcp_option(tcp_options, 'NOP')

            else:
              tcp_sport = tcp_dport = tcp_seq = tcp_ack = tcp_dataofs = tcp_reserved = tcp_flags = tcp_window = tcp_chksum = tcp_urgptr = tcp_mss = tcp_wscale = tcp_nop_count = None

            raw_data = packet.original.hex()
            writer.writerow([
                i + 1, eth_dst, eth_src, eth_type,
                  ip_version, ip_ihl, ip_tos, ip_len, ip_id, ip_flags,
                  ip_frag, ip_ttl, ip_proto, ip_chksum,
                  ip_src, ip_dst, tcp_sport, tcp_dport, tcp_seq,
                  tcp_ack, tcp_dataofs, tcp_reserved, tcp_flags,
                  tcp_window, tcp_chksum, tcp_urgptr, tcp_mss, tcp_wscale, tcp_nop_count,
                  raw_data
            ])

      return f"The file has been converted to csv successfully."
    except Exception as e:
      return f"An error occurred: {e}"

a = NetworkPreprocessing()
#a.convert_pcap_csv(pcap_file='/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/network.csv')
#a.delete_data(file_path='/content/drive/MyDrive/hybrid_IDS/network.csv')

In [ ]:
class PcapFileProcessing:
  def __init__(self) -> None:
    pass



  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."


  def date_time_columns(self, file_path, csv_file):
    try:
        df = pd.read_csv(file_path, dtype=str)
        df.columns = df.columns.str.strip()
        if 'timestamp' not in df.columns:
            return "Error: The 'timestamp' column is missing from the file."

        df['timestamp'] = df['timestamp'].astype(str).str.strip()

        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', dayfirst=True)

        if df['timestamp'].isnull().all():
            return "All values in 'timestamp' are invalid and could not be converted."

        # Extract date and time
        df['date'] = df['timestamp'].dt.strftime('%d/%m/%Y')
        df['time'] = df['timestamp'].dt.strftime('%H:%M:%S')

        # Save the result
        df.to_csv(csv_file, index=False)

        return "Successfully converted 'timestamp' to 'date' and 'time'."

    except Exception as e:
        return f"An error occurred during conversion: {e}"





  def convert_log_csv(self, log_file, csv_file):
    try:
      print("Start converting to csv file...")
      with open(log_file, "r") as logfile:
        log_data = logfile.read()
      pattern = re.compile(
          r'(?P<timestamp>\d{2}/\d{2}/\d{4}-\d{2}:\d{2}:\d{2}\.\d+)\s+\[\*\*\]\s+\[(?P<sid>[^]]+)\]\s+(?P<signature>.*?)\s+\[\*\*\]\s+\[Classification:\s+(?P<classification>.*?)\]\s+\[Priority:\s+(?P<priority>\d+)\]\s+\{(?P<protocol>\w+)\}\s+(?P<src_ip>\d+\.\d+\.\d+\.\d+):(?P<src_port>\d+)\s+->\s+(?P<dst_ip>\d+\.\d+\.\d+\.\d+):(?P<dst_port>\d+)'
          )
      with open(csv_file, "w", newline="") as csvfile:
        fieldnames = ['timestamp', 'sid', 'signature', 'classification',
                      'priority', 'protocol','src_ip', 'src_port', 'dst_ip', 'dst_port']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for match in pattern.finditer(log_data):
          writer.writerow(match.groupdict())

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"


  def convert_json_csv(self, json_file, csv_file):
    try:
      print("Start converting JSON to CSV...")
      with open(json_file, 'r') as f:
        data = [json.loads(line) for line in f if line.strip()]

      fieldnames = set()
      for row in data:
          fieldnames.update(row.keys())

      fieldnames = list(fieldnames)

      with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(fieldnames)

        for row in data:
          writer.writerow(row.get(field, '') for field in fieldnames)

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"



a = PcapFileProcessing()
#a.convert_log_csv( log_file="/content/drive/MyDrive/hybrid_IDS/fast.log",
#                  csv_file="/content/drive/MyDrive/hybrid_IDS/alerts.csv")

#a.convert_json_csv(json_file='/content/drive/MyDrive/hybrid_IDS/eve.json',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/eve.csv')

#a.delete_data( file_path='/content/drive/MyDrive/hybrid_IDS/eve.csv')
a.date_time_columns(file_path='/content/drive/MyDrive/hybrid_IDS/eve.csv',
                    csv_file='/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv')